In [13]:
import numpy as np
from theano import function
import theano.tensor.slinalg
import theano.tensor as T

from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
import timeit

rng = np.random

In [34]:
class mDA(object):
    def __init__(self, input):
        self.input = input
        
        self.W = T.dmatrix("W")
        self.Xh = T.dmatrix("Xh")
        
        self.output = T.tanh(T.dot(self.W, self.Xh))
        
    
    def get_mapping(self, pr):
        # X=[X;ones(1,size(X,2))];
        X = T.concatenate([self.input, T.ones((1, self.input.shape[1]))], axis=0)
        
        #d=size(X,1);
        d = X.shape[0]
        
        #q=[ones(d-1,1).*(1-p); 1];
        q = T.concatenate([T.ones((d-1,1)) * (1-pr), T.ones((1,1))], axis=0)
        
        #S=X*X’;
        S = T.dot(X, X.T)
        
        #Q=S.*(q*q’);
        Q = S * T.dot(q, q.T)
        
        #Q(1:d+1:end)=q.*diag(S);
        Q -= (T.eye(Q.shape[0]) * Q.diagonal())
        Q += T.eye(Q.shape[0]) * T.diagonal(q*S.diagonal())
        
        #P=S.*repmat(q’,d,1);
        P = S * T.extra_ops.repeat(q.T, d, 0)
        
        #W=P(1:end-1,:)/(Q+1e-5*eye(d));
        
        A = Q + 10**-5*T.eye(d)
        B = P
        
        self.W = T.slinalg.solve(A.T,B.T)[:-1,:]
        self.Xh = T.tanh(T.dot(self.W, X))
        
        return self.W, self.Xh
    
    def predict(self, X, W):
        X = T.concatenate([X, T.ones((1, X.shape[1]))], axis=0)
        #return T.tanh(T.dot(mapping, X))
        return T.tanh(T.dot(W, X))


class mSDA(object):
    def __init__(self, input, l, pr):
        self.input = input
        self.l = l
        self.pr = pr
        
        self.mDA_layers = []
        self.W_layers = []
        
        for i in xrange(self.l):
            if i==0:
                layer_input = self.input
            else:
                layer_input = self.mDA_layers[-1].output
        
            mda = mDA(layer_input)
            
            self.mDA_layers.append(mda)
    
    def train_fns(self):
        fns = []
        for mda in self.mDA_layers:
            W, Xh = mda.get_mapping(self.pr)
            train_mda = function(
                [mda.input],
                [W, Xh]
            )
            
            fns.append(train_mda)
        
        return fns
    
    def predict(self, X):
        Xhs = [X]
        
        for i in xrange(self.l):
            mda = self.mDA_layers[i]
            W = self.W_layers[i]

            x2 = T.dmatrix('x')
            W2 = T.dmatrix('w')

            y = mda.predict(x2, W2)
            
            
            pred = function(
                [x2, W2],
                y
            )
            
            print "W: ", W.shape
            
            Xh = pred(Xhs[i], W)
            
            print "Pred: ", Xh.shape
            
            Xhs.append(Xh)
        
        return Xhs[-1]

In [41]:
x = T.dmatrix('x')
# Dataset ficticio formado por 1000 ejemplos de dimension 500

samples = 25000
n_test = 400
features = 2000

D_train = (rng.randint(0,2, (samples, features)), rng.randint(0,2,(samples, )))
D_test = (rng.randint(0,2, (n_test, features)), rng.randint(0,2,(n_test, )))

D_all = np.vstack([D_train[0], D_test[0]])


l = 5
pr = 0.7


In [42]:
msda = mSDA(x, l, pr)
training_fns = msda.train_fns()

In [43]:
start_time = timeit.default_timer()

inputs = [D_all]

i = 1
for fn in training_fns:
    print "Training layer ", i
    W, Xh = fn(inputs[-1].T)
    
    inputs.append(Xh.T)
    
    msda.W_layers.append(W)
    
    i += 1
    
end_time = timeit.default_timer()

print "Entrenado en %.2fm" % ((end_time - start_time)/60.)


Training layer  1
Training layer  2
Training layer  3
Training layer  4
Training layer  5
Entrenado en 4.42m


In [44]:
X_new = msda.predict(D_test[0].T)

W:  (2000, 2001)
Pred:  (2000, 400)
W:  (2000, 2001)
Pred:  (2000, 400)
W:  (2000, 2001)
Pred:  (2000, 400)
W:  (2000, 2001)
Pred:  (2000, 400)
W:  (2000, 2001)
Pred:  (2000, 400)


In [45]:


tuned_parameters = [{'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]


clf1 = GridSearchCV(SVC(), tuned_parameters, n_jobs=4)
clf2 = GridSearchCV(SVC(), tuned_parameters, n_jobs=4)

In [46]:
print D_test[0].shape
print X_new.shape
print D_test[1].shape

(400, 2000)
(2000, 400)
(400,)


In [47]:
clf1 = clf1.fit(D_test[0], D_test[1])
print clf1.best_score_

clf2 = clf2.fit(X_new.T, D_test[1])
print clf2.best_score_



0.5025
0.505
